In [1]:
# Variables from config.json
temperature = 0.0
verbose = True
use_langfuse = True
model_name = "o3-2025-04-16"
username = "Shreyashgupta5"
code_link = "https://huggingface.co/spaces/Shreyashgupta5/ai_agents_course"
api_base_url = "https://agents-course-unit4-scoring.hf.space"